In [0]:
import keras
import numpy as np 
import os
from PIL import Image
import random
import imageio
from keras.preprocessing import sequence
from google.colab import drive
import matplotlib.pyplot as plt
from keras import backend as K

In [0]:
'''
This function does one-hot encoding of input images based on emotion (7 classes)
Sixth character of each file name corresponds to one of the seven emotions
'''
def label_image(name):
  letter = name[5] 

  if letter == 'W':
    return np.array([1,0,0,0,0,0,0])

  elif letter == 'L':
    return np.array([0,1,0,0,0,0,0])

  elif letter == 'E':
    return np.array([0,0,1,0,0,0,0])

  elif letter == 'A':
    return np.array([0,0,0,1,0,0,0])

  elif letter == 'F':
    return np.array([0,0,0,0,1,0,0])

  elif letter == 'T':
    return np.array([0,0,0,0,0,1,0])

  else:
    return np.array([0,0,0,0,0,0,1])
    

In [0]:
'''
The three different file paths are for three different parameters used for classification: Amplitude, Group Delay and Amplitude + Group Delay respectively.
Uncomment the appropriate line to run the network for the required input data
'''
drive.mount('/content/drive')
#DIR = '/content/drive/My Drive/Amplitude'
#DIR = '/content/drive/My Drive/Group_Delay'
#DIR = '/content/drive/My Drive/Amp_GPdelay'




In [0]:
'''
This function splits input of 535 images randomly into nearly 70:30 of training:testing data

'''
def train_test_split():                   # returns the splitted output and input data set with labelling 
  test_data = []
  train_data = []
  image_train = []
  labelled_train = []
  image_test = []
  labelled_test = []

  for img in os.listdir(DIR):
    label = label_image(img)              # returns a output class for each trainin image 
    path = os.path.join(DIR,img)
    img = Image.open(path)

    a = random.random()
    if a<=0.7:                            # 70:30 split between train and test 
      image_train.append(np.array(img))
      labelled_train.append(label)
    else:
      image_test.append(np.array(img))
      labelled_test.append(label)

    
  train_data.append(image_train)
  train_data.append(labelled_train)
  test_data.append(image_test)
  test_data.append(labelled_test)         # test data has 2 arrays, one for test images and the other one having correspoding output vector 

  return [train_data,test_data]



In [0]:
'''
Loading train and test data
'''
[train,test] = train_test_split()


In [0]:
'''
Unpacking of loaded data into lists_for giving input to network
'''

trainimages = np.array(train[0])
trainlabels = np.array(train[1])

testimages = np.array(test[0])
testlabels = np.array(test[1])


In [0]:
'''
Custom functions to calculate recall, precision, F1_score
'''
def recall(y_true, y_pred):
  true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
  possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
  recall = true_positives / (possible_positives + K.epsilon())
  return recall

def precision(y_true, y_pred):
  true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
  predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
  precision = true_positives / (predicted_positives + K.epsilon())
  return precision

def f1_score(y_true, y_pred):
  p = precision(y_true, y_pred)
  r = recall(y_true, y_pred)
  score = (2*p*r)/(p+r + K.epsilon())
  return score 


In [0]:
'''
Network Model

'''
import numpy as np
from keras.layers import Activation, Dense, Conv2D, Flatten, MaxPool2D, LSTM, Bidirectional, Dropout, BatchNormalization, Reshape
from keras.models import Sequential

model = Sequential() 


model.add(Conv2D(32, (5,5) ,padding='same',input_shape = (656, 875, 3), activation = 'relu'))
model.add(BatchNormalization(axis=-1))  
model.add(MaxPool2D(pool_size = 2, strides = 2, padding = 'valid'))

model.add(Conv2D(64, (5,5) , padding='same', activation = 'relu'))
model.add(BatchNormalization(axis=-1))
model.add(MaxPool2D(pool_size = 2, strides = 2, padding = 'valid'))
#Reduce third dimension and flatten 3d matrix to 2d matrix with time and frequency axis 
model.add(Conv2D(1, (1,1) , padding='same', activation = 'linear'))  
model.add(BatchNormalization(axis=-1))

#flatten 2d matrix to 1d array of time distributed sequence
model.add(Flatten())   
print(model.output_shape)

model.add(Dropout(0.5))

#fully connected layer
model.add(Dense(50, activation = 'relu'))
#print(model.output_shape)
model.add(Reshape((50,1)))
#print(model.output_shape)

#BLSTM
model.add(Bidirectional(LSTM(200, input_shape = (50,1), return_sequences=False), merge_mode = "concat"))

model.add(Dense(7,activation = 'softmax'))
print(model.summary())
#print(model.output_shape)

In [0]:
'''
Compiling and training and validating model
'''
model.compile('adam', loss = 'binary_crossentropy', metrics=['accuracy',precision,recall, f1_score])  
model.fit(trainimages, trainlabels, batch_size=50, epochs=10, validation_data=[testimages, testlabels])



In [0]:
'''
Evaluate the model
'''
loss, acc, prec, rec, f1 = model.evaluate(testimages, testlabels, verbose=0)